In [1]:
import os
import pandas as pd
import numpy as np
import mikeio
from datetime import datetime

import warnings
warnings.filterwarnings("ignore")

In [11]:
# Define folder directories
root = r'C:\62802426 Sg Pahang-Sg Kuantan NAWABS\Models\MBASIN\Pahang' # Model folder
global_data_dir = r'C:\62802426 Sg Pahang-Sg Kuantan NAWABS\Models\CBA\Pahang\Global_data_SgPahang.xlsx' # Global data path
out_dir = r'C:\62802426 Sg Pahang-Sg Kuantan NAWABS\Models\CBA\Pahang' # Directory to export results

res = [x for x in os.listdir(root)]
model = [s.strip(' - Result Files') for s in res]
scenario = [s.strip('.mhydro') for s in model]

for i in range(len(scenario)):
    try:
        # Define output file path and name
        #out_dir = fdir # When enabled, this option saves the results to the model "Results" folder instead
        output = os.path.join(out_dir,f"SgPahang_{scenario[i]}_indicators.txt")
        
        # Read global data
        global_data = pd.read_excel(global_data_dir, sheet_name=f'WTP_{scenario[i].split("-")[-1]}')

        # Water disruptions loss (Potable water)
        p_loss_per_month = 41313 #RM 
        p_loss_per_day = p_loss_per_month/30 #RM 

        # Water disruptions loss (Non-potable water)
        np_loss_per_month = 50000 #RM 
        np_loss_per_day = np_loss_per_month/30 #RM 

        # Irrigation losses calculation
        def irr_losses(irr_area):
            irr_ton = 3*irr_area # ton/ha
            irr_losses_per_year = irr_ton*3200 #RM
            return irr_losses_per_year

        # Read scenario master list
        ex_node = pd.read_excel(global_data_dir, sheet_name='Scenario')

        # Locate result file directory
        fdir = os.path.join(root,res[i]) 

        # Locate DFS0 result file 
        fn = [x for x in os.listdir(fdir) if x.endswith('.dfs0')]
        fn = [file for file in fn if 'Basin' in file]

        # Read data
        df = mikeio.read(os.path.join(fdir,fn[0])).to_dataframe()
        df.columns = [s.replace("  ", " ") for s in df.columns.to_list()] # MIKEIO sometimes read spaces as double space

        # Scenario manager: Filters away unused nodes in global data
        ex_node_1 = ex_node.copy()
        ex_node_1 = ex_node_1[ex_node_1['Scenario']==scenario[i]]
        ex_node_1 = ex_node_1.loc[:, (ex_node_1 != 1).all(axis=0)]
        global_data = global_data[~global_data['User Node'].isin(ex_node_1.columns)]
        global_data = global_data.reset_index()

        # Define list of irrigation nodes
        global_data_irr = global_data[global_data['Irrigation Node']==1]
        irr_list = global_data_irr["User Node"].to_list()
            
        # Define list of resevoir nodes
        global_data_dam = global_data[global_data['Reservoir Node']==1]
        dam_list = global_data_dam["User Node"].to_list()

        # Define list of ALL WTP nodes
        global_data_WTP = global_data[global_data['Reservoir Node']==0]
        WTP_list = global_data_WTP["User Node"].to_list()

        # Define list of non-potable WTP nodes
        global_data_npWTP = global_data[global_data['Non-potable Node']==1]
        npWTP_list = global_data_npWTP["User Node"].to_list()
        
        # Initiate new indicators.txt
        with open(output, 'w') as file:
            file.write("name;aggregation;item;value;timeseriesFile;durationFile;description;;\n")

        # WRI calculation for WTP
        # Results extraction
        results = global_data_WTP['WRI_dfs0_ref'].str.replace('\xa0', '')
        results = results.to_list()
        df1 = df.copy()
        df1 = df1[results]
        df1.columns = global_data_WTP['User Node'].to_list()

        # Calculate inflow
        df1_inflow = df1*3600*24/1000000

        # Calculate WRA
        df1_WRA = df1_inflow.copy()

        for k in range(len(df1_WRA.columns)):
            df1_WRA[df1_WRA.columns[k]] = df1_WRA[df1_WRA.columns[k]].rolling(window=7, min_periods=7).sum()

        # Calculate WRI
        df1_WRI = df1_WRA.copy()
        for k in range(len(df1_WRI.columns)):
            df1_WRI[df1_WRI.columns[k]] = df1_WRI[df1_WRI.columns[k]]/global_data['WRC'][k]

        # Calculate average WRI 
        df1_WRI_avg = df1_WRI.mean().to_frame()

        # Write to indicators.txt
        with open(output, 'a') as file:
            for k in range(len(df1_WRI_avg)):
                # Create the string to write
                line = f"WRI;Details;{df1_WRI_avg.index[k]};{df1_WRI_avg[0][k]};;;\n"
                # Write the string to the file
                file.write(line)

        # WRI calculation for Resevoirs
        # Results extraction (Storage Volume)
        results = global_data_dam['WRI_dfs0_ref'].str.replace('\xa0', '')
        results = results.to_list()
        df1_vol = df.copy()
        df1_vol = df1_vol[results]
        df1_vol.columns = global_data_dam['User Node'].to_list()

        # Results extraction (Net flow to node)
        results = [item.replace('Stored volume', 'Net flow to node') for item in results]
        df1_q = df.copy()
        df1_q = df1_q[results]
        df1_q.columns = global_data_dam['User Node'].to_list()


        # Convert stored volume to active volume
        for x in range(len(global_data_dam)):
            df1_vol[df1_vol.columns[x]] = df1_vol[df1_vol.columns[x]]-(global_data_dam['Dead storage'].values[x]*1000000)

        # Offset date by 6 days
        df1_vol = df1_vol.shift(6)

        # Ensure dataframe is truncated to original end date after offset
        df1_vol = df1_vol.truncate(after=df1_q.index[-1])

        # Calculate inflow
        df1_inflow = df1_q*3600*24/1000000

        # Calculate WRA
        df1_WRA = df1_inflow.copy()

        for k in range(len(df1_WRA.columns)):
            df1_WRA[df1_WRA.columns[k]] = df1_WRA[df1_WRA.columns[k]].rolling(window=7, min_periods=7).sum() + (df1_vol[df1_vol.columns[k]]/1000000)

        # Calculate WRI
        df1_WRI = df1_WRA.copy()
        for k in range(len(df1_WRI.columns)):
            df1_WRI[df1_WRI.columns[k]] = df1_WRI[df1_WRI.columns[k]]/global_data_dam['WRC'].values[k]

        # Calculate average WRI 
        df1_WRI_avg = df1_WRI.mean().to_frame()

        # Write to indicators.txt
        with open(output, 'a') as file:
            for k in range(len(df1_WRI_avg)):
                # Create the string to write
                line = f"WRI;Details;{df1_WRI_avg.index[k]};{df1_WRI_avg[0][k]};;;\n"
                # Write the string to the file
                file.write(line)

        # No of Deficit Days (WATER AVAILABILITY) and Deficit loss (ECONOMIC) calculation
        # Results extraction
        results = global_data['deficit_dfs0_ref']
        results = results.to_list()
        df2 = df.copy()
        df2 = df2[results]
        df2.columns = global_data['User Node'].to_list()

        # Initiate empty dataframe to store deficit days
        deficitdays=[]

        for k in range(len(df2.columns)):
            
            # If node is a reservoir node, calculate threshold = 40% of FSL volume (first TS) and count days where volume < threshold
            if df2.columns[k] in dam_list:
                threshold = df2[df2.columns[k]][0]*0.4
                count = (df2[df2.columns[k]] < threshold).sum()
                deficitdays.append(count)
            
            # Else count for days where deficit is greater than 0
            else:
                count = (df2[df2.columns[k]] > 0).sum()
                deficitdays.append(count)

        df3 = pd.DataFrame({'Node': df2.columns, 'Deficit_Days': deficitdays})
        df3['Deficit_loss'] = np.nan

        for k in range(len(df3)):

            # If node is a irrigation node, 
            if df3['Node'][k] in irr_list:
                # Resample to yearly max
                df2_year = df2.resample('Y').max()
                df2_year = df2_year[[df3['Node'][k]]]

                # Calculate irrigation losses per year
                irr_losses_per_year = irr_losses(global_data['Irrigation area'][k])
                
                # If yearly max > 0, indicates there are losses that year, loss calculations are based on per year basis
                count = (df2_year[df3['Node'][k]] > 0).sum()
                df3.loc[k, 'Deficit_loss'] = count * irr_losses_per_year
            
            # If node is a non-potable WTP node
            elif df3['Node'][k] in npWTP_list:
                df3.loc[k, 'Deficit_loss'] = df3.loc[k, 'Deficit_Days'] * np_loss_per_day

            # Else node is a regular potable WTP node
            else:
                df3.loc[k, 'Deficit_loss'] = df3.loc[k, 'Deficit_Days'] * p_loss_per_day

        # Write to indicators.txt
        with open(output, 'a') as file:
            for k in range(len(df3)):
                # Create the string to write
                line = f"Water Availability;Details;{df3['Node'][k]};{df3['Deficit_Days'][k]};;;\n"
                # Write the string to the file
                file.write(line)

                line = f"Economic;Details;{df3['Node'][k]};{df3['Deficit_loss'][k]};;;\n"
                line = line.replace("nan", "") # Reservoir nodes does not require calculations for losses, np.nan was used in place
                file.write(line)

        # Environmental Flow Sufficiency
        # Results extraction
        results = global_data['eflow_dfs0_ref']
        results = results.to_list()
        df4 = df.copy()
        df4 = df4[results]
        df4.columns = global_data['User Node'].to_list()

        # Initiate empty dataframe to sore deficit days
        higherdays_ratio=[]

        for k in range(len(df4.columns)):
            count = (df4[df4.columns[k]] > global_data['eflow'][k]).sum()
            ratio = (count/len(df4[df4.columns[k]]))*100
            higherdays_ratio.append(ratio)
        df5 = pd.DataFrame({'Node': df4.columns, 'Higher_Days_ratio': higherdays_ratio})

        # Write to indicators.txt
        with open(output, 'a') as file:
            for k in range(len(df5)):
                # Create the string to write
                line = f"Environmental Flow Sufficiency;Details;{df5['Node'][k]};{df5['Higher_Days_ratio'][k]};;;\n"
                # Write the string to the file
                file.write(line)

        # Social segment calculations
        # Read population data
        pop = pd.read_excel(global_data_dir,sheet_name="Population")
        for j in range(1,len(pop.columns)):
            district = pop.columns[j]
            population = pop[pop['Scenario']== scenario[i]]
            population = population[district].values[0]

            # Read WTP data
            WTP = pd.read_excel(global_data_dir,sheet_name="Social")
            
            # Filter away rows not present in WTP list
            WTP = WTP[WTP['User Node'].isin(WTP_list)]

            # Group WTP for selected district
            total_district_design_cap_ratio = WTP['Design Capacity'].sum()

            # Calculate populate served based on ratio to total design capacity
            WTP['population_served'] = population * (WTP['Design Capacity'] / WTP['Design Capacity'].sum())
            WTP = WTP.dropna(subset=['User Node'])

            with open(output, 'a') as file:
                for l in range(len(WTP)):
                    # Create the string to write
                    line = f"Social;Details;{WTP['User Node'].to_list()[l]};{WTP['population_served'].to_list()[l]};;;\n"
                    # Write the string to the file
                    file.write(line)
        
        print(f'[{datetime.now().strftime("%H:%M:%S")}] Successfully generated {output}')

    except Exception as e:
        if os.path.exists(output):
            os.remove(output)
        #print(f'[{datetime.now().strftime("%H:%M:%S")}] ERROR: Generation failed for {scenario[i]}\n{e}')

print(f'[{datetime.now().strftime("%H:%M:%S")}] End of script.')

c:\Users\mymoduser\AppData\Local\Programs\Python\Python310\lib\site-packages\mikeio\dataset\_dataset.py:180: UserWarning: Duplicate item name: E236|Flow. Renaming to E236|Flow_2
  warnings.warn(
c:\Users\mymoduser\AppData\Local\Programs\Python\Python310\lib\site-packages\mikeio\dataset\_dataset.py:180: UserWarning: Duplicate item name: E236|Seepage loss. Renaming to E236|Seepage loss_2
  warnings.warn(
c:\Users\mymoduser\AppData\Local\Programs\Python\Python310\lib\site-packages\mikeio\dataset\_dataset.py:180: UserWarning: Duplicate item name: E236|Evaporation loss. Renaming to E236|Evaporation loss_2
  warnings.warn(
c:\Users\mymoduser\AppData\Local\Programs\Python\Python310\lib\site-packages\mikeio\dataset\_dataset.py:180: UserWarning: Duplicate item name: E236|Water depth. Renaming to E236|Water depth_2
  warnings.warn(
c:\Users\mymoduser\AppData\Local\Programs\Python\Python310\lib\site-packages\mikeio\dataset\_dataset.py:180: UserWarning: Duplicate item name: E236|Mass balance. Rena

[15:05:01] Successfully generated C:\62802426 Sg Pahang-Sg Kuantan NAWABS\Models\CBA\Pahang\SgPahang_CC2-WD2-WS1_indicators.txt


c:\Users\mymoduser\AppData\Local\Programs\Python\Python310\lib\site-packages\mikeio\dataset\_dataset.py:180: UserWarning: Duplicate item name: E236|Flow. Renaming to E236|Flow_2
  warnings.warn(
c:\Users\mymoduser\AppData\Local\Programs\Python\Python310\lib\site-packages\mikeio\dataset\_dataset.py:180: UserWarning: Duplicate item name: E236|Seepage loss. Renaming to E236|Seepage loss_2
  warnings.warn(
c:\Users\mymoduser\AppData\Local\Programs\Python\Python310\lib\site-packages\mikeio\dataset\_dataset.py:180: UserWarning: Duplicate item name: E236|Evaporation loss. Renaming to E236|Evaporation loss_2
  warnings.warn(
c:\Users\mymoduser\AppData\Local\Programs\Python\Python310\lib\site-packages\mikeio\dataset\_dataset.py:180: UserWarning: Duplicate item name: E236|Water depth. Renaming to E236|Water depth_2
  warnings.warn(
c:\Users\mymoduser\AppData\Local\Programs\Python\Python310\lib\site-packages\mikeio\dataset\_dataset.py:180: UserWarning: Duplicate item name: E236|Mass balance. Rena

[15:05:21] Successfully generated C:\62802426 Sg Pahang-Sg Kuantan NAWABS\Models\CBA\Pahang\SgPahang_CC2-WD2-WS2_indicators.txt
[15:05:21] End of script.
